In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import numpy as np
import requests
import gmaps
import os
import json
import seaborn as sns
import sys
sys.path.insert(0, 'Data')
from config import g_key
from shapely.geometry import Point
%matplotlib inline

#Store Part I results into DataFrame
#Load the data to a DataFrame
accident_data = pd.read_csv("Data/accident_data.csv", encoding="utf-8")
accident_data.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [2]:
# Data cleanup

# Filters out dates containing 2016, 2017, & 2019 -- too much data in dataset to handle with laptops
accident_data_revised = accident_data[~accident_data["Start_Time"].str.contains("2016")]
accident_data_revised = accident_data[~accident_data["Start_Time"].str.contains("2017")]
accident_data_revised = accident_data[~accident_data["Start_Time"].str.contains("2019")]


In [4]:
# Splits the Start_Time column into individual columns
accident_data_revised[["Date", "Time"]] = accident_data_revised["Start_Time"].str.split(expand=True)

In [5]:
# Renames columns to be more readable
accident_data_revised = accident_data_revised.rename(columns={"Start_Lat":"Lat", "Start_Lng":"Lng", "Weather_Condition":"Weather"})

# Filters and rearranges dataset to display most useful columns
accident_data_revised = accident_data_revised[["Date", "Time", "Lat", "Lng", "City", "State", "County", "Weather", "Temperature(F)", "Severity"]]

# Iterates through all column data to find NaN values and adds them to a list
accident_data_revised.columns[accident_data_revised.isna().any()].tolist()
# Output: ['City', 'Weather', 'Temperature(F)']
accident_data_revised.head(10)

,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity
0,2016-02-08,05:46:00,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,36.9,3
1,2016-02-08,06:07:59,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,37.9,2
2,2016-02-08,06:49:27,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,36.0,2
3,2016-02-08,07:23:34,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,35.1,3
4,2016-02-08,07:39:07,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,36.0,2
5,2016-02-08,07:44:26,40.100590,-82.925194,Westerville,OH,Franklin,Light Rain,37.9,3
6,2016-02-08,07:59:35,39.758274,-84.230507,Dayton,OH,Montgomery,Overcast,34.0,2
7,2016-02-08,07:59:58,39.770382,-84.194901,Dayton,OH,Montgomery,Overcast,34.0,3
8,2016-02-08,08:00:40,39.778061,-84.172005,Dayton,OH,Montgomery,Mostly Cloudy,33.3,2
9,2016-02-08,08:10:04,40.100590,-82.925194,Westerville,OH,Franklin,Light Rain,37.4,3


In [6]:
# Combine Lat/Lng and City/State
accident_data_revised['Coordinates'] = accident_data_revised['Lat'].astype(str).str.zfill(2) + ', ' + accident_data_revised['Lng'].astype(str).str.zfill(3)

accident_data_revised['Location'] = accident_data_revised['City'] + ", " + accident_data_revised['State']
accident_data_revised.head()

,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Coordinates,Location
0,2016-02-08,05:46:00,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,36.9,3,"39.865147, -84.058723","Dayton, OH"
1,2016-02-08,06:07:59,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,37.9,2,"39.92805900000001, -82.831184","Reynoldsburg, OH"
2,2016-02-08,06:49:27,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,36.0,2,"39.063148, -84.032608","Williamsburg, OH"
3,2016-02-08,07:23:34,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,35.1,3,"39.747753, -84.20558199999998","Dayton, OH"
4,2016-02-08,07:39:07,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,36.0,2,"39.627781, -84.188354","Dayton, OH"


In [30]:
accident_data_revised['frequency'] = accident_data_revised['Coordinates'].map(accident_data_revised['Coordinates'].value_counts())
accident_data_revised.head()

,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Coordinates,Location,frequency
0,2016-02-08,05:46:00,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,36.9,3,"39.865147, -84.058723","Dayton, OH",1
1,2016-02-08,06:07:59,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,37.9,2,"39.92805900000001, -82.831184","Reynoldsburg, OH",1
2,2016-02-08,06:49:27,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,36.0,2,"39.063148, -84.032608","Williamsburg, OH",3
3,2016-02-08,07:23:34,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,35.1,3,"39.747753, -84.20558199999998","Dayton, OH",201
4,2016-02-08,07:39:07,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,36.0,2,"39.627781, -84.188354","Dayton, OH",24


In [31]:
accident_data_by_coordinates = accident_data_revised.nlargest(100,'frequency')
accident_data_by_coordinates.head()

,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Coordinates,Location,frequency
833,2016-06-22,04:31:35,37.808498,-122.366852,San Francisco,CA,San Francisco,Mostly Cloudy,55.9,3,"37.808498, -122.366852","San Francisco, CA",570
1167,2016-06-24,19:12:11,37.808498,-122.366852,San Francisco,CA,San Francisco,Clear,70.0,3,"37.808498, -122.366852","San Francisco, CA",570
1237,2016-06-25,07:17:37,37.808498,-122.366852,San Francisco,CA,San Francisco,Clear,60.1,3,"37.808498, -122.366852","San Francisco, CA",570
1322,2016-06-26,08:16:07,37.808498,-122.366852,San Francisco,CA,San Francisco,Partly Cloudy,63.0,3,"37.808498, -122.366852","San Francisco, CA",570
1392,2016-06-27,09:48:41,37.808498,-122.366852,San Francisco,CA,San Francisco,Clear,64.9,3,"37.808498, -122.366852","San Francisco, CA",570


In [11]:
#Configure gmaps
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations
locations = accident_data_revised[['Location']]
figure_layout = {
    'width': '400px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout)
#accident_data_revised.plot(figsize=(20,10))

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='400px'))

In [12]:
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#Configure gmaps
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable
markers = gmaps.marker_layer('Coordinates')
# Add the layer to the map
fig.add_layer(markers)
fig


In [ ]:
#Add Heatmap layer to map.
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=maxhumidity,
                                    point_radius=2)
fig.add_layer(heatmap_layer)
fig

In [10]:
# Combining Lattitude and Longitude to create coordinates:
accident_data_revised['Coordinates'] = accident_data_revised[['Lat', 'Lng']].values.tolist()
accident_data_revised.head()

,ID,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Coordinates,Location
0,A-1,2016-02-08,05:46:00,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,36.9,3,"[39.865147, -84.058723]","Dayton, OH"
1,A-2,2016-02-08,06:07:59,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,37.9,2,"[39.92805900000001, -82.831184]","Reynoldsburg, OH"
2,A-3,2016-02-08,06:49:27,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,36.0,2,"[39.063148, -84.032608]","Williamsburg, OH"
3,A-4,2016-02-08,07:23:34,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,35.1,3,"[39.747753, -84.20558199999998]","Dayton, OH"
4,A-5,2016-02-08,07:39:07,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,36.0,2,"[39.627781, -84.188354]","Dayton, OH"


In [16]:
locations = accident_data_revised[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig
accident_data_revised.head()

AttributeError: module 'gmaps' has no attribute 'datasets'

In [11]:
#Change the coordinates to a geoPoint
accident_data_revised['Coordinates'] = accident_data_revised['Coordinates'].apply(Point)
accident_data_revised.head()

,ID,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Coordinates,Location
0,A-1,2016-02-08,05:46:00,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,36.9,3,POINT (39.865147 -84.058723),"Dayton, OH"
1,A-2,2016-02-08,06:07:59,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,37.9,2,POINT (39.92805900000001 -82.83118399999999),"Reynoldsburg, OH"
2,A-3,2016-02-08,06:49:27,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,36.0,2,POINT (39.063148 -84.032608),"Williamsburg, OH"
3,A-4,2016-02-08,07:23:34,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,35.1,3,POINT (39.747753 -84.20558199999998),"Dayton, OH"
4,A-5,2016-02-08,07:39:07,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,36.0,2,POINT (39.627781 -84.188354),"Dayton, OH"


In [12]:
# Convert the count df to geodf
accident_data_revised = geopandas.GeoDataFrame(accident_data_revised, geometry='Coordinates')
accident_data_revised.head()

,ID,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Coordinates,Location
0,A-1,2016-02-08,05:46:00,39.865147,-84.058723,Dayton,OH,Montgomery,Light Rain,36.9,3,POINT (39.86515 -84.05872),"Dayton, OH"
1,A-2,2016-02-08,06:07:59,39.928059,-82.831184,Reynoldsburg,OH,Franklin,Light Rain,37.9,2,POINT (39.92806 -82.83118),"Reynoldsburg, OH"
2,A-3,2016-02-08,06:49:27,39.063148,-84.032608,Williamsburg,OH,Clermont,Overcast,36.0,2,POINT (39.06315 -84.03261),"Williamsburg, OH"
3,A-4,2016-02-08,07:23:34,39.747753,-84.205582,Dayton,OH,Montgomery,Mostly Cloudy,35.1,3,POINT (39.74775 -84.20558),"Dayton, OH"
4,A-5,2016-02-08,07:39:07,39.627781,-84.188354,Dayton,OH,Montgomery,Mostly Cloudy,36.0,2,POINT (39.62778 -84.18835),"Dayton, OH"


In [59]:
# Change the coordinates to a geoPoint
accident_data_revised['Coordinates'] = accident_data_revised['Coordinates'].apply(Point)
accident_data_revised.head()

NameError: name 'Point' is not defined